File Formats

---

Les dades s'emmagatzemen normalment en fitxers, que es construeixen segons un format de fitxer específic. Els formats de fitxer estandarditzats sovint són compatibles amb Python mitjançant un mòdul concret. En aquest capítol, comentem alguns dels formats de fitxer més comuns i els mòduls que els admeten.

---

## Valors separats per comes (Comma-Separated Values CSV)

Els valors separats per comes (CSV) són el format de fitxer de text més comú que s'utilitza per importar i exportar dades a i des de fulls de càlcul i bases de dades. El format general diu que cada línia conté un registre (un registre és una entitat completa), enumerant cadascun dels camps del registre en un ordre específic, separant els camps per comes. La primera línia del fitxer pot consistir o no en noms per als camps del fitxer CSV.

El codi següent carrega i mostra el contingut d'un fitxer CSV típic que vaig adjuntar al curs.

In [8]:
fp = open( "pc_inventory.csv" )
print( fp.read().strip() )
fp.close()

FileNotFoundError: [Errno 2] No such file or directory: 'pc_inventory.csv'

Malauradament, el format CSV no està estandarditzat i els diferents paquets de programari solen utilitzar implementacions lleugerament diferents dels fitxers CSV. No obstant això, al llarg dels anys, les diferents convencions utilitzades pels principals paquets de programari han convergit cap a una mena d'estàndard, que s'implementa en el mòdul `csv` de Python. El mòdul admet "dialectes" de formats CSV per gestionar fitxers de diferents fonts.

En lloc d'utilitzar el mòdul `csv`, si heu de tractar amb un format CSV excèntric que el mòdul no admet, podeu provar de dissenyar la vostra pròpia interpretació de les línies del fitxer utilitzant expressions regulars. També podeu provar de dissenyar el vostre propi dialecte. Cap de les dues opcions és molt atractiva.

### CSV `reader()`

El mòdul `csv` conté una funció `reader()` que proporciona accés a un fitxer CSV. La funció `reader()` obté un identificador de fitxer com a argument i retorna un iterador que us permet obtenir les línies del fitxer, com una llista amb cadascun dels camps com a element de la llista. Hauríeu de deixar el fitxer obert mentre hi accediu amb `reader()`.

In [7]:
from csv import reader

fp = open( "pc_inventory.csv", newline='' )
csvreader = reader( fp )
for line in csvreader:
    print( line )
fp.close()

FileNotFoundError: [Errno 2] No such file or directory: 'pc_inventory.csv'

La documentació de Python recomana que si utilitzeu `reader()` en un fitxer (i això és el que normalment feu), especifiqueu un argument `newline=''` com a argument addicional en obrir el fitxer a dalt). Això és necessari en cas que alguns dels camps de text del fitxer CSV continguin caràcters de nova línia.

`reader()` també pren arguments addicionals. Els més comuns són `delimiter=<character>`, que indica quin `<character>` es col·loca entre diferents camps (per defecte és "`,`"), i `quotechar=<character>`, que indica quin `<character>` s'utilitza per incloure cadenes. amb (per defecte és "`"`").

### CSV `writer()`

Writing a CSV file is just a little bit harder than reading one. You create a file handle to a file that you open for writing ("`w`" mode), and use it as an argument when you call the `writer()` function from the `csv` module. The object that is returned from the `writer()` call has a method `writerow()` that you can call with a list of fields, that it then writes to the output file in CSV format.

The call to `writer()` can get the same arguments as the call to `reader()` can get, including specifying a `delimiter` and a `quotechar`. Moreover, you can a supply a `quoting=<quotemethod>` argument, that supports the following methods of quoting:

- `csv.QUOTE_ALL`, which encloses every field in quotation characters
- `csv.QUOTE_MINIMAL`, which only encloses fields in quotation characters if it is absolutely necessary (this is the default)
- `csv.QUOTE_NONNUMERIC`, which encloses fields in quotation characters if they are not integers or floats
- `csv.QUOTE_NONE`, which encloses no fields in quotation characters

Enclosing a string within quotation characters is generally only needed if the string contains exceptional characters, such as newlines or the same character that is used as delimiter. 

Escriure un fitxer CSV és una mica més difícil que llegir-ne un. Creeu un identificador de fitxer per a un fitxer que obriu per escriure ("mode `w`") i l'utilitzeu com a argument quan crideu la funció `writer()` des del mòdul `csv`. L'objecte que es retorna de la crida `writer()` té un mètode `writerow()` que podeu cridar amb una llista de camps, que després escriu al fitxer de sortida en format CSV.

La crida a `writer()` pot obtenir els mateixos arguments que la crida a `reader()`, inclosa l'especificació d'un `delimitador` i un `quotechar`. A més, podeu proporcionar un argument `quoting=<quotemethod>`, que admeti els mètodes següents de citar:

- `csv.QUOTE_ALL`, que inclou tots els camps entre cometes
- `csv.QUOTE_MINIMAL`, que només inclou els camps entre cometes si és absolutament necessari (aquest és el valor predeterminat)
- `csv.QUOTE_NONNUMERIC`, que inclou els camps entre cometes si no són nombres enters o flotants
- `csv.QUOTE_NONE`, que no inclou camps entre cometes

En general, només cal incloure una cadena entre cometes si la cadena conté caràcters excepcionals, com ara noves línies o el mateix caràcter que s'utilitza com a delimitador.

In [6]:
from csv import writer

fp = open( "pc_writetest.csv", "w", newline='' )
csvwriter = writer( fp )
csvwriter.writerow( ["MOVIE", "RATING"] )
csvwriter.writerow( ["Monty Python and the Holy Grail", 8] )
csvwriter.writerow( ["Monty Python's Life of Brian", 8.5] )
csvwriter.writerow( ["Monty Python's Meaning of Life", 7] )
fp.close()

**Exercici**: després d'utilitzar el codi anterior per crear el fitxer "pc_writetest.csv", obriu-lo i utilitzeu `reader()` per llistar-ne el contingut.

In [19]:
# Read what was written.
import csv

In [23]:
with open("pc_writetest.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=",")
    for row in reader:
        print(row)

csvfile

<_io.TextIOWrapper name='pc_writetest.csv' mode='r' encoding='cp1252'>

---

## Pickling

Suposem que voleu emmagatzemar una determinada estructura de dades en un fitxer, per exemple, una llista de tuples. Una manera de fer-ho és convertir les tuples en cadenes i escriure-les al fitxer, una línia per cada tupla. Quan més tard voleu reconstruir l'estructura de dades en un programa, llegiu el fitxer, desfeu les línies i reconstruïu la llista de tuples. Com us podeu imaginar, això abasta una quantitat considerable de codi bastant difícil.

Afortunadament, no cal escriure aquest codi. Python ofereix una solució per emmagatzemar estructures de dades en fitxers, tant l'estructura com el contingut, que s'anomena "pickling". Podeu escriure tota l'estructura de dades al fitxer d'una vegada, si només obriu un fitxer *binari* per escriure, i truqueu la funció `dump()` des del mòdul `pickle` amb l'estructura de dades com primer argument i el controlador del fitxer com a segon argument.

In [2]:
from pickle import dump

cheeseshop = [ ("Roquefort", 12, 15.23), ("White Stilton", 25, 19.02), ("Cheddar", 5, 0.67) ]

fp = open( "pc_cheese.pck", "wb" )
dump( cheeseshop, fp )
fp.close()

print( "Cheeseshop was pickled" )

Cheeseshop was pickled


Per llegir el contingut d'un fitxer pickle, feu servir la funció `load()` del mòdul `pickle`. `load()` obté un identificador per al fitxer com a argument. No oblideu obrir el fitxer en mode binari.

In [3]:
from pickle import load

fp = open( "pc_cheese.pck", "rb" )
buffer = load( fp )
fp.close()

print( type( buffer ) )
print( buffer )

<class 'list'>
[('Roquefort', 12, 15.23), ('White Stilton', 25, 19.02), ('Cheddar', 5, 0.67)]


Com podeu veure, `load()` restaura l'estructura de dades completament.

El decapat funciona fins i tot per a les vostres classes:

In [4]:
from pickle import dump, load

class Point:
    def __init__( self, x, y ):
        self.x = x
        self.y = y
    def __repr__( self ):
        return "({},{})".format( self.x, self.y )
    
p = Point( 2, 5 )
fp = open( "pc_point.pck", "wb" )
dump( p, fp )
fp.close()

fp = open( "pc_point.pck", "rb" )
q = load( fp )
fp.close()

print( type( q ) )
print( q )

<class '__main__.Point'>
(2,5)


---

## JavaScript Object Notation (JSON)

La notació d'objectes JavaScript (JSON) és un format de fitxer que s'utilitza sovint en aplicacions modernes, en particular aquelles que es comuniquen mitjançant serveis web. És compatible amb molts idiomes (JavaScript entre ells, és clar). És similar al decapatge en el sentit que emmagatzema objectes a la memòria en fitxers, conservant la seva estructura. Una diferència amb el decapatge és que els fitxers JSON estan en format llegible per l'home.

El mòdul `json` funciona de manera equivalent al mòdul `pickle`, amb una funció `dump()` que escriu estructures de dades en un fitxer i una funció `load()` per carregar estructures de dades d'un fitxer. El fitxer ha de ser un fitxer de text i no un fitxer binari.

In [5]:
from json import dump, load

cheeseshop = [ ("Roquefort", 12, 15.23), ("White Stilton", 25, 19.02), ("Cheddar", 5, 0.67) ]

fp = open( "pc_cheese.json", "w" )
dump( cheeseshop, fp )
fp.close()

fp = open( "pc_cheese.json", "r" )
buffer = load( fp )
fp.close()

print( type( buffer ) )
print( buffer )

<class 'list'>
[['Roquefort', 12, 15.23], ['White Stilton', 25, 19.02], ['Cheddar', 5, 0.67]]


Les alternatives per a `dump()` i `load()` són les funcions `dumps()` i `loads()`, que no reben un argument de fitxer. En canvi, `dumps()` no obté cap argument de fitxer i només produeix una cadena que conté l'estructura de dades en format JSON, mentre que `loads()` obté una cadena com a argument en lloc d'un fitxer i carrega l'estructura de dades des de aquella string.

Aquestes funcions poden obtenir molts arguments opcionals que determinen com s'emmagatzemaran exactament les dades; per exemple, podeu establir l'argument `indent=` per a `dump()` i `dumps()` per determinar quin valor de sagnat s'utilitzarà, i podeu utilitzar arguments per ordenar les dades a l'abocament. Si vols saber més sobre això, consulta les referències.

Una debilitat del mòdul `json` és que només admet les estructures de dades estàndard de Python. Si voleu utilitzar-lo per emmagatzemar instàncies de classes fetes per vosaltres mateixos, heu de trobar una manera de convertir les vostres classes a estructures estàndard de Python. El mòdul `json` ofereix classes especials `JSONencoder` i `JSONdecoder` per ajudar-vos amb això. Va massa lluny per parlar d'aquests aquí.

---

## HTML and XML

HTML i XML són formats estàndard que s'utilitzen per mostrar informació a les pàgines web. Consten de fitxers de text llegibles, amb moltes instruccions sobre el format. És una tasca habitual per als miners de dades "rascar" dades de les pàgines web. Podeu utilitzar expressions regulars per a això, però si les pàgines web estan raonablement ben formatades, el mòdul "Beautiful Soup" us pot ajudar.

El mòdul Beautiful Soup s'anomena "bs4" a Python (naturalment, "bs3" va ser anterior, i pot ser que rebi més actualitzacions més endavant). Conté la classe `BeautifulSoup` que podeu utilitzar per carregar i interpretar fitxers HTML i XML. `bs4` no forma part del paquet estàndard de Python; l'heu d'instal·lar per separat, cosa que és una molèstia, tret que utilitzeu una eina anomenada "pip" que ve de sèrie amb Python 3.

Hi ha mòduls alternatius que us poden alleujar el dolor del web scrapping, especialment `lxml`, però Beautiful Soup sembla ser el més popular

---